In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-21 13:09:12.438344: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 13:09:12.463576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 13:09:12.463601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 13:09:12.464719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 13:09:12.471226: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4464737719208833591
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2100822016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4509028172416448216
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-21 13:09:13.420207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:13.450894: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:13.450932: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:14.170847: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:14.170894: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42,stratify=y_train)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(38751, 277)
(72794, 277)
X_train shape: (27125, 275)
X_val shape: (11626, 275)
y_train shape: (27125,)
y_val shape: (11626,)


In [4]:
from keras.layers import LSTM, Dropout, Dense,Flatten
from keras.optimizers import Nadam
from keras.losses import categorical_crossentropy  
from keras.utils import to_categorical
from keras import backend as K

def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed


# Initialize the model
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(X_train1.shape[1],X_train1.shape[2])))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True, name = 'last_lstm_layer'))
model.add(Flatten())
# Add a Dense layer
model.add(Dense(7, activation='softmax'))  # Assuming 10 classes for classification

# Compile the model with focal loss
model.compile(loss=focal_loss(gamma=2.0), optimizer=Nadam(), metrics=['accuracy'])
model.summary()

2024-03-21 13:09:15.927103: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:15.927182: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:15.927201: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:15.927311: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 13:09:15.927327: I external/local_xla/xla/stream_executor

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 275, 64)           16896     
                                                                 
 lstm_1 (LSTM)               (None, 275, 64)           33024     
                                                                 
 lstm_2 (LSTM)               (None, 275, 64)           33024     
                                                                 
 last_lstm_layer (LSTM)      (None, 275, 64)           33024     
                                                                 
 flatten (Flatten)           (None, 17600)             0         
                                                                 
 dense (Dense)               (None, 7)                 123207    
                                                                 
Total params: 239175 (934.28 KB)
Trainable params: 23917

In [5]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [6]:
class_weights

array([0.25926669, 2.33393564, 2.33273134, 2.33273134, 2.33393564,
       2.33273134])

In [7]:

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',min_delta=0.001, restore_best_weights=True)
history = model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)




Epoch 1/50


2024-03-21 13:09:26.235279: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-21 13:09:26.857391: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f939c28aa50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-21 13:09:26.857431: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-21 13:09:26.864436: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711026566.921973   36037 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


848/848 [==============================] - 62s 69ms/step - loss: 0.0278 - accuracy: 0.9018 - val_loss: 0.0157 - val_accuracy: 0.9471
Epoch 2/50
848/848 [==============================] - 67s 79ms/step - loss: 0.0137 - accuracy: 0.9495 - val_loss: 0.0106 - val_accuracy: 0.9610
Epoch 3/50
848/848 [==============================] - 66s 78ms/step - loss: 0.0101 - accuracy: 0.9611 - val_loss: 0.0091 - val_accuracy: 0.9646
Epoch 4/50
848/848 [==============================] - 60s 71ms/step - loss: 0.0085 - accuracy: 0.9671 - val_loss: 0.0088 - val_accuracy: 0.9675
Epoch 5/50
848/848 [==============================] - 66s 78ms/step - loss: 0.0071 - accuracy: 0.9721 - val_loss: 0.0073 - val_accuracy: 0.9718
Epoch 6/50
848/848 [==============================] - 59s 70ms/step - loss: 0.0061 - accuracy: 0.9747 - val_loss: 0.0068 - val_accuracy: 0.9741
Epoch 7/50
848/848 [==============================] - 54s 64ms/step - loss: 0.0056 - accuracy: 0.9777 - val_loss: 0.0070 - val_accuracy: 0.9768
Epo

In [8]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58951  2493  6725  2923  1277   425]


In [9]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = model.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 65s 29ms/step
Confusion Matrix:
[[58427     0    21    66   342    95]
 [    0  2492     1     0     0     0]
 [   11     0  6698     3    10     3]
 [   65     0    13  2800    14    31]
 [   92     0     8     1  1174     2]
 [    2     0     0     2     0   421]]
F1 Score: 0.9896530765044852


In [10]:
model.save('./models/lstmmodel.h5')
